# Merged RAG Notebook — Road Safety Intervention GPT (NRSH 2025)

This notebook is the **final merged version** you asked for. It combines:

- Chunked retrieval (better FAISS retrieval using text splitting)
- Topic-wise synthesis and structured output (Problem → IRC Clauses → Topic-wise Interventions → Steps → Cost → Compliance)
- Strict metadata citations (Option A)
- Interactive `process_query` function (Option 1 format)

---


## Step 0 — Quick notes
- Put your `knowledge_base.json` in the same folder as this notebook.
- Do NOT hardcode secrets in the notebook. Set your Hugging Face token as an environment variable:
    export HF_TOKEN="<your_token>"
  or set HF_TOKEN in your session before running the LLM-loading cell.
- Install required libraries (run once):
    %pip install langchain langchain-community sentence-transformers faiss-cpu transformers torch accelerate huggingface_hub


## Step 1 — Imports & Setup


In [1]:
import os
import json
import time
from typing import List, Dict, Any, Tuple

# Langchain + embeddings + FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

# Transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Avoid tokenizer parallelism warnings
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Config
JSON_FILE_PATH = "knowledge_base.json"
EMBEDDING_MODEL = "BAAI/bge-large-en-v1.5"
HF_MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
HF_TOKEN = os.getenv('HF_TOKEN', None)  # set this in your environment
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if getattr(torch.backends, 'mps', False) and torch.backends.mps.is_available() else "cpu")

CHUNK_SIZE = 600
CHUNK_OVERLAP = 120
TOP_K = 5

print(f"Device: {DEVICE}")


/Users/ram/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: mps


## Step 2 — Load & Normalize `knowledge_base.json`
This cell can handle both the IRC table-style items and the intervention-style items you showed earlier.


In [2]:
def load_and_prepare_documents(file_path: str) -> List[Document]:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"{file_path} not found. Place your knowledge base JSON in the same directory."
        )

    doc_list = []

    for item in data:

        # -------------------------------------------------
        # CASE 1 — Gold-standard merged format
        # -------------------------------------------------
        if 'intervention_name' in item and 'content' in item:
            page_content = (
                f"Intervention/Standard: {item['intervention_name']}\n\n"
                f"{item['content']}"
            )

            metadata = item.get('metadata', {})
            final_metadata = dict(metadata)
            final_metadata.setdefault('id', item.get('id', 'N/A'))
            final_metadata.setdefault(
                'source_reference',
                metadata.get('source_reference', item.get('source_reference', 'N/A'))
            )

        # -------------------------------------------------
        # CASE 2 — IRC table-style (s_no + data)
        # -------------------------------------------------
        elif 's_no' in item and 'data' in item:
            page_content = (
                f"Standard Type: {item.get('type', 'N/A')}\n"
                f"Specification: {item.get('data', 'N/A')}"
            )

            final_metadata = {
                'id': item.get('s_no', 'N/A'),
                'source_reference': item.get('code', 'IRC'),
                'irc_clause': f"{item.get('code', 'IRC')}, Clause {item.get('clause', 'N/A')}",
                'type': 'Standard',
                'category': item.get('category', 'N/A'),
            }

        # -------------------------------------------------
        # CASE 3 — Intervention entries (intervention + description)
        # -------------------------------------------------
        elif 'intervention' in item and 'description' in item:
            content_text = (
                f"{item['description']}\n"
                f"When to Apply: {item.get('when_to_apply', 'N/A')}\n"
                f"Why it Works: {item.get('why_it_works', 'N/A')}"
            )

            page_content = (
                f"Intervention: {item['intervention']}\n\n"
                f"{content_text}"
            )

            final_metadata = {
                'id': item.get('id', 'N/A'),
                'source_reference': item.get('source', 'N/A'),
                'type': 'Intervention',
                'category': item.get('category', 'N/A'),
            }

        # -------------------------------------------------
        # CASE 4 — Fallback
        # -------------------------------------------------
        else:
            text = json.dumps(item, ensure_ascii=False)
            page_content = f"Document:\n{text[:1000]}"

            final_metadata = {
                'id': item.get('id', 'N/A'),
                'source_reference': item.get('source', 'N/A')
            }

        # -------------------------------------------------
        # Append final doc
        # -------------------------------------------------
        doc_list.append(
            Document(page_content=page_content, metadata=final_metadata)
        )

    return doc_list


# Load documents
documents = load_and_prepare_documents(JSON_FILE_PATH)
print(f"Loaded {len(documents)} documents")
print(documents[0].page_content[:400])


Loaded 240 documents
Intervention/Standard: STOP Sign

The 'STOP' sign, used on Minor Roads intersecting Major Roads, requires vehicles to stop before entering and proceed only when safe. It is octagonal with a red background, a white border, and "STOP" written centrally in white. Installed on the left side of the approach, it should be placed close to the stop line, typically 1.5 m in advance, without impairing visib


## Step 3 — Chunking & Vector Store


In [3]:
# Chunk the combined documents but preserve metadata
chunker = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks, metas = [], []
for doc in documents:
    splits = chunker.split_text(doc.page_content)
    for s in splits:
        chunks.append(s)
        # copy metadata and keep id/source_reference
        m = dict(doc.metadata)
        m.setdefault('topic', 'General')
        metas.append(m)

print(f"Total chunks created: {len(chunks)}")

# Create vector store
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = FAISS.from_texts(chunks, embeddings, metadatas=metas)
print("Vector store built.")


Total chunks created: 330


/var/folders/7q/cwlrl67n1p3fsjh24d6dk5r00000gn/T/ipykernel_22969/3033526530.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)


Vector store built.


In [ ]:
import os
os.environ["HF_TOKEN"] = "" 


## Step 4 — LLM Loader (no hardcoded tokens)


In [ ]:
def load_llm(
    model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
    token: str = ""   # ← put your HF token directly here
):
    """
    Loads the Llama 3.2 model using the token directly passed into the function.
    """

    if token == "YOUR_TOKEN_HERE" or not token:
        raise ValueError("❌ ERROR: Please replace 'YOUR_TOKEN_HERE' with your actual HF token.")

    print(f"Loading model: {model_name} on device: {DEVICE} ...")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        token=token
    )

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto" if DEVICE == "cuda" else None,
        torch_dtype=torch.bfloat16 if DEVICE == "cuda" else None,
        use_auth_token=token
    )

    # Fix pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Create pipeline
    llm = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=900,
        do_sample=False
    )

    print("✅ LLM loaded successfully.")
    return llm, tokenizer


# Load model
llm, tokenizer = load_llm()

if not llm:
    print("LLM not loaded.")


Loading model: meta-llama/Llama-3.2-3B-Instruct on device: mps ...


/Users/ram/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.51s/it]
Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ LLM loaded successfully.


## Step 5 — Intent Detection


In [5]:
def detect_intent_simple(query: str) -> str:
    q = query.lower()
    if 'summarize' in q or 'summary' in q:
        return 'request_summary'
    if any(x in q for x in ['cost', 'estimate', 'price', 'how much']):
        return 'cost_estimate'
    if any(x in q for x in ['fix', 'intervention', 'solution', 'what should i do', 'how to fix']):
        return 'find_intervention'
    if any(x in q for x in ['standard', 'specification', 'clause', 'irc', 'rule', 'compliance']):
        return 'find_standard'
    if any(x in q for x in ['compare', 'difference between', 'vs ', 'v/s']):
        return 'compare_interventions'
    if any(x in q for x in ['quiz', 'test me']):
        return 'request_quiz'
    return 'ask_question'


## Step 6 — Citation Formatter


In [6]:
def format_citation(metadata: Dict[str, Any]) -> str:
    # Strict metadata citation preference: 'irc_clause' -> 'source_reference' + 'id' -> fallback
    if not metadata:
        return '[Source: N/A]'
    if metadata.get('irc_clause'):
        return f"[{metadata['irc_clause']}]"
    parts = []
    if metadata.get('source_reference') and metadata['source_reference'] != 'N/A':
        parts.append(str(metadata['source_reference']))
    if metadata.get('id') and metadata['id'] != 'N/A':
        parts.append(str(metadata['id']))
    if parts:
        return '[' + ', '.join(parts) + ']'
    return '[Source: N/A]'


## Step 7 — Synthesis Prompt & Generation
This cell creates the topic-wise + structured prompt (Option 1) and invokes the LLM. If `llm` is None, it returns the retrieval results for inspection.


In [7]:
def synthesize_topic_wise(llm, tokenizer, retrieved_chunks: List[Dict[str, Any]], query: str, intent: str):
    # ----------------------------------------------
    # Build Context Blocks
    # ----------------------------------------------
    blocks = []
    citations = []

    for chunk in retrieved_chunks:
        meta = chunk.metadata if hasattr(chunk, "metadata") else chunk.get("metadata", {})
        text = chunk.page_content if hasattr(chunk, "page_content") else chunk.get("text", "")

        cite = format_citation(meta)
        citations.append(cite)

        block = (
            f"---\n"
            f"Citation: {cite}\n"
            f"Topic: {meta.get('topic', 'N/A')}\n"
            f"Metadata: {json.dumps(meta)}\n"
            f"Content: {text}\n"
        )
        blocks.append(block)

    context_text = "\n".join(blocks)

    # ----------------------------------------------
    # Build Prompt Template
    # ----------------------------------------------
    template = f"""
You are an Expert Road Safety Analyst. Use ONLY the provided context blocks to answer. Do NOT hallucinate.
Produce a TOPIC-WISE and STRUCTURED response with these exact sections:

### 1) Problem Interpretation

### 2) Applicable IRC Clauses / Sources

### 3) Topic-wise Recommended Interventions

### 4) Why This Works

### 5) Step-by-Step Fix Guide

### 6) Estimated Cost (label as ESTIMATE if not in context)

### 7) Compliance Check

### 8) Final Summary

Context Blocks:
{context_text}

User Query:
{query}

Notes:
- Every factual claim must include inline citations from the context (e.g., [IRC:67-2022, Clause 12.3] or [Source_ID]).
- If information is missing, say: "Cannot answer from provided knowledge base."
"""

    # ----------------------------------------------
    # If LLM Not Loaded → Only Show Retrieved Context
    # ----------------------------------------------
    if not llm:
        return "LLM not loaded. Showing retrieved context only.", citations

    # ----------------------------------------------
    # Generate Output From LLM
    # ----------------------------------------------
    output = llm(template, num_return_sequences=1)
    raw = output[0]["generated_text"]

    # Some pipelines echo the prompt; strip if needed
    if raw.startswith(template):
        answer = raw[len(template):].strip()
    else:
        answer = raw.strip()

    # Remove duplicate citations but preserve order
    unique_citations = list(dict.fromkeys(citations))

    return answer, unique_citations


## Step 8 — Process Query (Top-level function)


In [8]:
def process_query(query: str, top_k: int = TOP_K):
    print("\n" + "="*60)
    print(f"Query: {query}")
    
    # Detect intent
    intent = detect_intent_simple(query)
    print(f"Detected intent: {intent}\n")

    # Retrieve top-k chunks
    retrieved = vector_store.similarity_search(query, k=top_k)
    print(f"Retrieved {len(retrieved)} chunks.\n")

    # Get synthesized answer
    answer, citations = synthesize_topic_wise(llm, tokenizer, retrieved, query, intent)

    # Print answer
    print("--- ANSWER ---\n")
    print(answer)

    # Print citations
    print("\n--- CITATIONS ---")
    for c in citations:
        print(c)

    print("\n" + "="*60 + "\n")


## Step 9 — Next steps & utilities
- If you'd like, I can:
  - Add a small example `knowledge_base.json` with 6–8 representative entries so you can run the notebook immediately.
  - Add a Dockerfile and resource guidance for running Llama 3.2 3B locally.
  - Convert this notebook into a single `.py` module with CLI and unit tests.

Tell me which of those you'd like next; I'll add it directly into the notebook.


In [16]:
process_query("What interventions can I use for speeding in a school zone?")


Query: What interventions can I use for speeding in a school zone?
Detected intent: find_intervention

Retrieved 5 chunks.

--- ANSWER ---

- If the answer is not a simple "yes" or "no", provide a structured response.

### 1) Problem Interpretation

Speeding in school zones is a significant concern, as it poses a risk to the safety of children and pedestrians. The problem can be attributed to various factors, including inadequate infrastructure, lack of enforcement, and insufficient speed limit enforcement.

### 2) Applicable IRC Clauses / Sources

According to IRC:99-2018 - Clause 3.1.1.1, speed humps are a recommended traffic calming measure for reducing vehicle speeds in school zones.

### 3) Topic-wise Recommended Interventions

Based on the provided context, the following interventions can be used to address speeding in school zones:

*   Speed Humps in School Zones (IRC:99-2018 - Clause 3.1.1.1)
*   School Zone Enforcement Cameras (MoRTH Inspired, int-school_20)
*   Safe School 

In [9]:
process_query("My road markings are faded and not retro-reflective. What's the rule for that?")


Query: My road markings are faded and not retro-reflective. What's the rule for that?
Detected intent: find_standard

Retrieved 5 chunks.

--- ANSWER ---

- If the answer is not clear, say: "Cannot answer from provided knowledge base."

### 1) Problem Interpretation

The problem at hand is faded and non-retro-reflective road markings, which can lead to reduced visibility and increased risk of accidents, especially at night or in low-light conditions.

### 2) Applicable IRC Clauses / Sources

The problem is related to the standards for road markings, specifically Clause 4.2 of IRC:35-2015, which addresses the maintenance and replacement of traffic lane line markings, and Clause 2.7 of IRC:35-2015, which outlines the requirements for retro-reflective sheeting on road signs.

### 3) Topic-wise Recommended Interventions

Based on the provided context, the recommended interventions for faded and non-retro-reflective road markings are:

* Lane Marking Refurbishment (as per PWD Inspired, int